# Transfer Learning Assignment
반갑습니다 여러분. 과제를 맡은 김준호입니다. TL 세션에서 다룬 Fine-tuning과 Domain Adaptation을 직접 구현해 봅시다.

Office-31은 여러 실습에서 자주 등장하는 이미지 데이터셋입니다. Amazon, Webcam, DSLR 세 개의 도메인으로 구성되어 있는데요. 

총 31개 클래스 (키보드, 마우스, 모니터 등)의 사무용품 이미지가 있고,각 도메인마다 같은 클래스가 포함되어 있지만 도메인마다 이미지 특성은 다릅니다. 

그래서 이런 transfer learning 실습에 적합하다고 볼 수 있습니다.

In [ ]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import time
from torchvision import models
torch.cuda.set_device(0)

amazon을 source로, webcam을 target data로 이용해 봅시다.

In [ ]:
data_folder = 'OFFICE31'
batch_size = 32
n_class = 31
domain_src, domain_tar = 'amazon', 'webcam'

source와 target domain에 대한 DataLoader를 생성하고 load해 줍시다.

In [ ]:
def load_data(root_path, domain, batch_size, phase):
    transform_dict = {
        'src': transforms.Compose(
        [transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ]),
        'tar': transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ])}
    data = datasets.ImageFolder(root=os.path.join(root_path, domain), transform=transform_dict[phase])
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=phase=='src', drop_last=phase=='tar', num_workers=4)
    return data_loader

In [ ]:
src_loader = load_data(data_folder, domain_src, batch_size, phase='src')
tar_loader = load_data(data_folder, domain_tar, batch_size, phase='tar')
print(f'Source data number: {len(src_loader.dataset)}')
print(f'Target data number: {len(tar_loader.dataset)}')

# Fine-tuning

Pre-trained ResNet50을 기반으로 한 간단한 TransferModel을 정의합니다. 여기서 모델 구조는 유지하되 마지막 fc layer만 새롭게 학습되도록 구성합니다.

In [ ]:
class TransferModel(nn.Module):
    def __init__(self,
                base_model : str = 'resnet50',
                pretrain : bool = True,
                n_class : int = 31):
        super(TransferModel, self).__init__()
        self.base_model = base_model
        self.pretrain = pretrain
        self.n_class = n_class
        if self.base_model == 'resnet50':
            self.model = # TODO: pre-trained model 불러오기
            n_features = self.model.fc.in_features
            fc = # TODO: 새로운 fc layer를 정의
            # TODO: 모델의 fc layer를 새로운 fc로 교체
        else:
            # Use other models you like, such as vgg or alexnet
            pass
        self.model.fc.weight.data.normal_(0, 0.005)
        self.model.fc.bias.data.fill_(0.1)

    def forward(self, x):
        return self.model(x)
    
    def predict(self, x):
        return self.forward(x)

모델이 정상적으로 작동하는지 random tensor로 테스트해 봅시다.

In [ ]:
model = TransferModel().cuda()
RAND_TENSOR = torch.randn(1, 3, 224, 224).cuda()
output = model(RAND_TENSOR)
print(output)
print(output.shape)

## Finetune ResNet-50

Office-31 dataset은 validation set이 따로 없으므로, validation set으로 target domain을 이용해 줍시다.

fine-tuning을 위한 학습 및 평가 함수를 정의합니다.
학습은 source domain에서 수행하고 target domain에서 검증합니다.

In [ ]:
dataloaders = {'src': src_loader,
               'val': tar_loader,
               'tar': tar_loader}
n_epoch = 100
criterion = nn.CrossEntropyLoss()
early_stop = 20

In [ ]:
def finetune(model, dataloaders, optimizer):
    since = time.time()
    best_acc = 0
    stop = 0
    for epoch in range(0, n_epoch):
        stop += 1
        # You can uncomment this line for scheduling learning rate
        # lr_schedule(optimizer, epoch)
        for phase in ['src', 'val', 'tar']:
            if phase == 'src':
                model.train()
            else:
                model.eval()
            total_loss, correct = 0, 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.cuda(), labels.cuda()
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'src'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                preds = torch.max(outputs, 1)[1]
                if phase == 'src':
                    loss.backward()
                    optimizer.step()
                total_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
            epoch_loss = total_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct.double() / len(dataloaders[phase].dataset)
            print(f'Epoch: [{epoch:02d}/{n_epoch:02d}]---{phase}, loss: {epoch_loss:.6f}, acc: {epoch_acc:.4f}')
            if phase == 'val' and epoch_acc > best_acc:
                stop = 0
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'model.pkl')
        if stop >= early_stop:
            break
        print()
   
    time_pass = time.time() - since
    print(f'Training complete in {time_pass // 60:.0f}m {time_pass % 60:.0f}s')

이제 학습 파라미터들과 optimizer를 정의합니다.
간단하게 SGD optimizer를 사용하고 fc layer의 학습률을 다른 layer보다 10배 크게 설정합니다.

In [ ]:
param_group = []
learning_rate = 0.0001
momentum = 5e-4
for k, v in model.named_parameters():
    if not k.__contains__('fc'):
        param_group += [{'params': v, 'lr': learning_rate}]
    else:
        param_group += [{'params': v, 'lr': learning_rate * 10}]
optimizer = torch.optim.SGD(param_group, momentum=momentum)

## Train and test

In [ ]:
# TODO: fine-tuning function 호출하여 학습 진행

In [ ]:
def test(model, target_test_loader):
    model.eval()
    correct = 0
    len_target_dataset = len(target_test_loader.dataset)
    with torch.no_grad():
        for data, target in target_test_loader:
            data, target = data.cuda(), target.cuda()
            s_output = model.predict(data)
            pred = torch.max(s_output, 1)[1]
            correct += torch.sum(pred == target)
    acc = correct.double() / len(target_test_loader.dataset)
    return acc

In [ ]:
model.load_state_dict(torch.load('model.pkl'))
acc_test = test(model, dataloaders['tar'])
print(f'Test accuracy: {acc_test}')

여기까지가 fine-tuning 파트입니다. 실제 학습에서는 learning rate decay 같은 기법도 사용하지만, 이 과제에서는 그것이 핵심이 아니므로 생략합니다.

이제 같은 dataloader를 그대로 활용해서 domain adaptation 실험을 이어가봅시다.

# Domain Adaptation

Domain adaptation의 핵심 구조는 fine-tuning과 매우 비슷하지만,
두 도메인 간 분포 차이를 줄이기 위한 loss function을 추가해야 합니다.

여기서는 MMD와 Coral loss를 사용해 두 도메인 간 분포 차이를 계산하는 loss function을 정의합니다. 

해당 loss를 이용할 수 있도록 새로운 모델 클래스를 정의하고 source의 feature와 label, 그리고 target feature를 모두 이용하도록 학습 스크립트를 수정해봅시다.

### Loss function
Domain Adaptation에서 가장 많이 사용되는 손실 함수는 MMD (Maximum Mean Discrepancy)입니다.

비교를 위해 또 다른 대표적인 손실 함수인 CORAL (CORrelation ALignment)도 함께 살펴봅니다.

#### MMD loss

In [ ]:
class MMD_loss(nn.Module):
    def __init__(self, kernel_type='rbf', kernel_mul=2.0, kernel_num=5):
        super(MMD_loss, self).__init__()
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        self.fix_sigma = None
        self.kernel_type = kernel_type

    def guassian_kernel(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
        n_samples = int(source.size()[0]) + int(target.size()[0])
        total = torch.cat([source, target], dim=0)
        total0 = total.unsqueeze(0).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_distance = ((total0-total1)**2).sum(2)
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul**i)
                          for i in range(kernel_num)]
        kernel_val = [torch.exp(-L2_distance / bandwidth_temp)
                      for bandwidth_temp in bandwidth_list]
        return sum(kernel_val)

    def linear_mmd2(self, f_of_X, f_of_Y):
        loss = 0.0
        delta = f_of_X.float().mean(0) - f_of_Y.float().mean(0)
        loss = delta.dot(delta.T)
        return loss

    def forward(self, source, target):
        if self.kernel_type == 'linear':
            return self.linear_mmd2(source, target)
        elif self.kernel_type == 'rbf':
            batch_size = int(source.size()[0])
            kernels = self.guassian_kernel(
                source, target, kernel_mul=self.kernel_mul, kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
            XX = torch.mean(kernels[:batch_size, :batch_size])
            YY = torch.mean(kernels[batch_size:, batch_size:])
            XY = torch.mean(kernels[:batch_size, batch_size:])
            YX = torch.mean(kernels[batch_size:, :batch_size])
            loss = torch.mean(XX + YY - XY - YX)
            return loss


#### CORAL loss

In [ ]:
def CORAL(source, target):
    d = source.size(1)
    ns, nt = source.size(0), target.size(0)

    # source covariance
    tmp_s = torch.ones((1, ns)).cuda() @ source
    cs = (source.t() @ source - (tmp_s.t() @ tmp_s) / ns) / (ns - 1)

    # target covariance
    tmp_t = torch.ones((1, nt)).cuda() @ target
    ct = (target.t() @ target - (tmp_t.t() @ tmp_t) / nt) / (nt - 1)

    # frobenius norm
    loss = (cs - ct).pow(2).sum().sqrt()
    loss = loss / (4 * d * d)

    return loss

### Model
여기서도 backbone으로는 ResNet-50을 사용합니다.
다만 이번에는 ResNet-50의 마지막 classifier layer를 제거한 feature extractor로 사용합니다.

In [ ]:
from torchvision import models
class ResNet50Fc(nn.Module):
    def __init__(self):
        super(ResNet50Fc, self).__init__()
        model_resnet50 = models.resnet50(pretrained=True)
        self.conv1 = model_resnet50.conv1
        self.bn1 = model_resnet50.bn1
        self.relu = model_resnet50.relu
        self.maxpool = model_resnet50.maxpool
        self.layer1 = model_resnet50.layer1
        self.layer2 = model_resnet50.layer2
        self.layer3 = model_resnet50.layer3
        self.layer4 = model_resnet50.layer4
        self.avgpool = model_resnet50.avgpool
        self.__in_features = model_resnet50.fc.in_features

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x

    def output_num(self):
        return self.__in_features

이제 Domain Adaptation을 위한 핵심 모델 클래스를 정의합니다.

ResNet-50을 기반으로 하되, bottleneck layer와 새로운 fc layer를 추가합니다.

중요한 점은 adapt_loss 함수로 우리가 정의한 MMD 또는 CORAL loss를 forward pass에서 함께 계산한다는 것입니다.

In [ ]:
class TransferNet(nn.Module):
    def __init__(self,
                 num_class, 
                 base_net='resnet50', 
                 transfer_loss='mmd', 
                 use_bottleneck=True, 
                 bottleneck_width=256, 
                 width=1024):
        super(TransferNet, self).__init__()
        if base_net == 'resnet50':
            # TODO: ResNet50 기반 feature extractor를 정의
        else:
            # Your own basenet
            return
        self.use_bottleneck = use_bottleneck
        self.transfer_loss = transfer_loss
        bottleneck_list = # TODO: bottleneck layer를 정의
        self.bottleneck_layer = nn.Sequential(*bottleneck_list)
        classifier_layer_list = # TODO: classifier layer를 정의
        self.classifier_layer = nn.Sequential(*classifier_layer_list)

        self.bottleneck_layer[0].weight.data.normal_(0, 0.005)
        self.bottleneck_layer[0].bias.data.fill_(0.1)
        for i in range(2):
            self.classifier_layer[i * 3].weight.data.normal_(0, 0.01)
            self.classifier_layer[i * 3].bias.data.fill_(0.0)

    def forward(self, source, target):
        source = self.base_network(source)
        target = self.base_network(target)
        source_clf = self.classifier_layer(source)
        if self.use_bottleneck:
            source = self.bottleneck_layer(source)
            target = self.bottleneck_layer(target)
        transfer_loss = self.adapt_loss(source, target, self.transfer_loss)
        return source_clf, transfer_loss

    def predict(self, x):
        features = self.base_network(x)
        clf = self.classifier_layer(features)
        return clf

    def adapt_loss(self, X, Y, adapt_loss):
        """Compute adaptation loss, currently we support mmd and coral

        Arguments:
            X {tensor} -- source matrix
            Y {tensor} -- target matrix
            adapt_loss {string} -- loss type, 'mmd' or 'coral'. You can add your own loss

        Returns:
            [tensor] -- adaptation loss tensor
        """
        if adapt_loss == 'mmd':
            mmd_loss = MMD_loss()
            loss = mmd_loss(X, Y)
        elif adapt_loss == 'coral':
            loss = CORAL(X, Y)
        else:
            # Your own loss
            loss = 0
        return loss

### Train
이제 Domain Adaptation 모델을 학습시켜 봅시다.

In [ ]:
transfer_loss = 'mmd'
learning_rate = 0.0001
transfer_model = TransferNet(n_class, transfer_loss=transfer_loss, base_net='resnet50').cuda()
optimizer = torch.optim.SGD([
    {'params': transfer_model.base_network.parameters()},
    {'params': transfer_model.bottleneck_layer.parameters(), 'lr': 10 * learning_rate},
    {'params': transfer_model.classifier_layer.parameters(), 'lr': 10 * learning_rate},
], lr=learning_rate, momentum=0.9, weight_decay=5e-4)
lamb = 10 # weight for transfer loss, it is a hyperparameter that needs to be tuned

학습 함수에서는 source 데이터와 label, target 데이터를 모두 사용해야 하므로, source와 target의 dataloader를 zip으로 묶어서 동시에 iterate합니다.

보통 두 도메인의 샘플 수는 다르기 때문에, 여러 epoch에 걸쳐 무작위로 잘 섞이면 전체 데이터를 충분히 학습할 수 있습니다.

In [ ]:
def train(dataloaders, model, optimizer):
    source_loader, target_train_loader, target_test_loader = dataloaders['src'], dataloaders['val'], dataloaders['tar']
    len_source_loader = len(source_loader)
    len_target_loader = len(target_train_loader)
    best_acc = 0
    stop = 0
    n_batch = min(len_source_loader, len_target_loader)
    for e in range(n_epoch):
        stop += 1
        train_loss_clf, train_loss_transfer, train_loss_total = 0, 0, 0
        model.train()
        for (src, tar) in zip(source_loader, target_train_loader):
            data_source, label_source = src
            data_target, _ = tar
            data_source, label_source = data_source.cuda(), label_source.cuda()
            data_target = data_target.cuda()

            optimizer.zero_grad()
            label_source_pred, transfer_loss = model(data_source, data_target)
            clf_loss = criterion(label_source_pred, label_source)
            loss = clf_loss + lamb * transfer_loss
            loss.backward()
            optimizer.step()
            train_loss_clf = clf_loss.detach().item() + train_loss_clf
            train_loss_transfer = transfer_loss.detach().item() + train_loss_transfer
            train_loss_total = loss.detach().item() + train_loss_total
        acc = test(model, target_test_loader)
        print(f'Epoch: [{e:2d}/{n_epoch}], cls_loss: {train_loss_clf/n_batch:.4f}, transfer_loss: {train_loss_transfer/n_batch:.4f}, total_Loss: {train_loss_total/n_batch:.4f}, acc: {acc:.4f}')
        if best_acc < acc:
            best_acc = acc
            torch.save(model.state_dict(), 'trans_model.pkl')
            stop = 0
        if stop >= early_stop:
            break

In [ ]:
# TODO: Domain Adaptation 모델을 학습시키는 함수를 호출하여 학습 진행

In [ ]:
transfer_model.load_state_dict(torch.load('trans_model.pkl'))
acc_test = test(transfer_model, dataloaders['tar'])
print(f'Test accuracy: {acc_test}')